<a href="https://colab.research.google.com/github/mclabs74/inference_nbs/blob/dev/huggingface/imdb_classification_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# main_branch
from pathlib import Path

def create_requirements_file():
    if not Path("requirements.txt").exists():
        requirements = ["transformers"]
        with open('requirements.txt', 'w') as filehandle:
            for listitem in requirements:
                filehandle.write('%s\n' % listitem)
    
create_requirements_file()

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
# !wget -q https://ml-inference.s3-us-west-2.amazonaws.com/hugging_face_imdb_training.zip
!wget https://ml-inference.s3-us-west-2.amazonaws.com/hugging_face_imdb_training.zip

In [ ]:
!unzip -n hugging_face_imdb_training.zip

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
model_path = Path("./model")
model_new = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer_new = DistilBertTokenizer.from_pretrained(model_path/"tokenizer")

In [ ]:
!pip install -q git+https://github.com/matthewchung74/inference_params.git

In [ ]:
import torch.nn.functional as F
import torch
from inference_params.inference_params import inference_test, FieldType, inference_predict

class_names =["negative", "positive"]

# text input will be the label for the app input
input = {"text input": FieldType.Text}
# result will be the label for the app output
output = {"result": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    text = params['text input']
    inputs = tokenizer_new(text, return_tensors="pt")
    labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
    outputs = model_new(**inputs, labels=labels)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)
    confidence, predicted_class = torch.max(probabilities, dim=1)
    return {"result": f"{class_names[predicted_class]}, confidence: {confidence.item()}"}

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    inference_test(predict_func=predict, params={'text input': 'loved that movie'})

In [ ]:
if in_colab():
    predict({'text input':'this movie is awesome'}) 

In [ ]:
if in_colab():
    predict({'text input':'this movie is very bad'})